# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

### Запишем функционал ошибки в данной задаче:
# $$ Q(w) = \sum_{i=1}^n (1 - y_i (<w, x_i> + w_0))_+ + \frac{\sum_{j=1}^m w^2_j}{2C} $$

### Частные производные:
# $$ \frac{\partial Q(w)}{\partial w_0} =  \sum_{i=1}^n (-y_i)[1 - y_i(<w, x_i> + w_0) > 0]$$
# $$ \frac{\partial Q(w)}{\partial w} =  \sum_{i=1}^n (-y_i x_i)[1 - y_i(<w, x_i> + w_0) > 0] + \frac{w}{C}$$


### Правило обновления весов:
# $$ w^{k+1} = w^k - \gamma \nabla_wQ(w^k) $$

### Используем SGD, фиксируем на каждом шаге индекс i, тогда:
# $$ w^{k+1}_0 = w^k_0 + \gamma y_i[1 - y_i(<w^k, x_i> + w^k_0) > 0] $$
# $$ w^{k+1} = w^k - \gamma (-y_i x_i[1 - y_i(<w^k, x_i> + w^k_0) > 0] + \frac{w^k}{C}) $$


In [2]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([self.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return self.w / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        return -1.0 if 1 - answer * self.f(x) > 0 else 0.0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k
        
            # w update            
            self.w -= step * (self.der_reg() + x * y * self.der_loss(x, y))
                
            # w_0 update              
            self.w0 -= step * y * self.der_loss(x, y)

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [3]:
model = MySVM(C=1)
model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.08399737 -0.11574877 -0.18893492  0.04294063 -0.02153957  0.23238928
  0.04839055 -0.12852017 -0.17796593 -0.07978025 -0.14086975  0.05867091
  0.41733011 -0.0767311  -0.27359771  0.08315033  0.30067117  0.35377186
  0.22580597 -0.08417844] -0.847676750841


In [4]:
my_svm_predictions = model.predict(X_test)

In [5]:
print(my_svm_predictions, len(my_svm_predictions), sum(my_svm_predictions))
print(y_test, len(y_test), sum(y_test))

[1 1 0 ..., 1 0 1] 2000 897
[1 0 1 ..., 1 0 1] 2000 991


In [6]:
accuracy_score(my_svm_predictions, y_test)

0.68300000000000005

## Сравним качество с sklearn LinearSVC

In [7]:
from sklearn.svm import LinearSVC

In [8]:
svc = LinearSVC(penalty='l2', loss='hinge', max_iter=10000, C=1)
svc.fit(X_train, y_train)
svc_predictions = svc.predict(X_test)

In [9]:
print(svc_predictions, len(svc_predictions), sum(svc_predictions))
print(y_test, len(y_test), sum(y_test))

[1 0 1 ..., 1 1 0] 2000 1020
[1 0 1 ..., 1 0 1] 2000 991


In [10]:
accuracy_score(svc_predictions, y_test)

0.79749999999999999